<center>
  <h2>Trabajo Laboratorio de Implementación III - MCD Virtual Cohorte 2022</h2>
  <h3>Forecasting para una Empresa de Consumo Masivo</h3>
  <h4>Experimento 14 AutoGluon</h4>
</center>

In [1]:
# !pip install autogluon
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

In [2]:
df = pd.read_csv("C:/Users/usuario/otros/Desktop/MCD/6 Labo III/Forecasting Problem/Datasets/df_ventas_product_fill_extreme_values_095.csv", header = None)
df.columns = ["timestamp", "item_id", "target"]
df.head(5)

,timestamp,item_id,target
0,2017-01-01,20001,934.77222
1,2017-02-01,20001,798.01620
2,2017-03-01,20001,1303.35771
3,2017-04-01,20001,1069.96130
4,2017-05-01,20001,1502.20132


In [3]:
df_product = pd.read_csv("C:/Users/usuario/otros/Desktop/MCD/6 Labo III/Forecasting Problem/Datasets/df_products_corr.csv", header = None)
df_product.columns = ["item_id", "cat1", "cat2", "cat3", "brand", "sku_size", 
                      "corr_20001", "corr_20002", "corr_20003", "corr_20005", "corr_20004", "corr_20009", "corr_20032", "corr_20006", "corr_20007", "corr_20010"]
df_product.head(5)

,item_id,cat1,cat2,cat3,brand,sku_size,corr_20001,corr_20002,corr_20003,corr_20005,corr_20004,corr_20009,corr_20032,corr_20006,corr_20007,corr_20010
0,20280,HC,ROPA LAVADO,Liquido,LIMPIEX,900,0.344376,0.251042,0.149971,0.056807,0.179661,0.107151,0.091045,-0.142871,0.511455,0.054308
1,20180,HC,ROPA LAVADO,Liquido,LIMPIEX,450,0.277107,0.164447,0.035649,0.454793,0.478442,0.356229,0.201214,-0.003315,0.772228,0.737458
2,20332,HC,ROPA LAVADO,Liquido,LIMPIEX,120,-0.620015,-0.316214,-0.103129,0.029887,0.088297,0.197564,0.070152,-0.304822,0.373020,0.510303
3,20222,HC,ROPA LAVADO,Liquido,LIMPIEX,450,0.403952,0.245037,0.294852,0.568689,0.475148,0.344290,0.123933,0.222539,0.442538,0.883702
4,20288,HC,ROPA LAVADO,Liquido,LIMPIEX,900,0.441335,0.129863,0.266306,0.254267,0.241912,0.499360,0.288560,-0.433734,0.179030,0.003566


In [4]:
df_related = pd.read_csv("C:/Users/usuario/otros/Desktop/MCD/6 Labo III/Forecasting Problem/Datasets/df_related_product_v2.csv", header = None)
related_cols = ['timestamp',
 'item_id',
 'month',
 'quarter',
 'days_in_month',
 'sundays',
 'saturdays',
 'lag1m',
 'lag2m',
 'lag3m',
 'lag4m',
 'lag5m',
 'lag6m',
 'lag7m',
 'lag8m',
 'lag9m',
 'lag10m',
 'lag11m',
 'lag12m',
 'delta1',
 'delta2',
 'delta3',
 'delta4',
 'delta5',
 'delta6',
 'delta7',
 'delta8',
 'delta9',
 'delta10',
 'delta11',
 'delta12',
 'monthly_percentage',
 'ipc',
 'dolar']
df_related.columns = related_cols
df_related.head(5)

,timestamp,item_id,month,quarter,days_in_month,sundays,saturdays,lag1m,lag2m,lag3m,...,delta6,delta7,delta8,delta9,delta10,delta11,delta12,monthly_percentage,ipc,dolar
0,2017-01-01,20001,1,1,31,5,4,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.310483,19.7,16.853182
1,2017-02-01,20001,2,1,28,4,4,934.77222,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.387267,19.5,16.425000
2,2017-03-01,20001,3,1,31,4,4,798.01620,934.77222,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.798738,18.4,16.062273
3,2017-04-01,20001,4,2,30,5,5,1303.35771,798.01620,934.77222,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.223120,17.5,15.921667
4,2017-05-01,20001,5,2,31,4,4,1069.96130,1303.35771,798.01620,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.141096,17.0,16.104762


In [5]:
df_extendido = pd.merge(df, df_related, how = "left", on = ["timestamp", "item_id"])
print(df_extendido.timestamp.max())
df_extendido.head(5)

2019-12-01


,timestamp,item_id,target,month,quarter,days_in_month,sundays,saturdays,lag1m,lag2m,...,delta6,delta7,delta8,delta9,delta10,delta11,delta12,monthly_percentage,ipc,dolar
0,2017-01-01,20001,934.77222,1,1,31,5,4,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.310483,19.7,16.853182
1,2017-02-01,20001,798.01620,2,1,28,4,4,934.77222,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.387267,19.5,16.425000
2,2017-03-01,20001,1303.35771,3,1,31,4,4,798.01620,934.77222,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.798738,18.4,16.062273
3,2017-04-01,20001,1069.96130,4,2,30,5,5,1303.35771,798.01620,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.223120,17.5,15.921667
4,2017-05-01,20001,1502.20132,5,2,31,4,4,1069.96130,1303.35771,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.141096,17.0,16.104762


In [6]:
data = TimeSeriesDataFrame.from_data_frame(df_extendido,
                                           id_column="item_id",
                                           timestamp_column="timestamp",
                                           static_features_df=df_product)
data.head()

target  month  quarter  days_in_month  sundays  \
item_id timestamp                                                        
20001   2017-01-01   934.77222      1        1             31        5   
        2017-02-01   798.01620      2        1             28        4   
        2017-03-01  1303.35771      3        1             31        4   
        2017-04-01  1069.96130      4        2             30        5   
        2017-05-01  1502.20132      5        2             31        4   

                    saturdays       lag1m       lag2m      lag3m      lag4m  \
item_id timestamp                                                             
20001   2017-01-01          4         NaN         NaN        NaN        NaN   
        2017-02-01          4   934.77222         NaN        NaN        NaN   
        2017-03-01          4   798.01620   934.77222        NaN        NaN   
        2017-04-01          5  1303.35771   798.01620  934.77222        NaN   
        2017-05-01          4  1069.96130  1303.35771  798.01620  934.77222   

                    ...  delta6  delta7  delta8  delta9  delta10  delta11  \
item_id timestamp   ...                                                     
20001   2017-01-01  ...     NaN     NaN     NaN     NaN      NaN      NaN   
        2017-02-01  ...     NaN     NaN     NaN     NaN      NaN      NaN   
        2017-03-01  ...     NaN     NaN     NaN     NaN      NaN      NaN   
        2017-04-01  ...     NaN     NaN     NaN     NaN      NaN      NaN   
        2017-05-01  ...     NaN     NaN     NaN     NaN      NaN      NaN   

                    delta12  monthly_percentage   ipc      dolar  
item_id timestamp                                                 
20001   2017-01-01      NaN            6.310483  19.7  16.853182  
        2017-02-01      NaN            5.387267  19.5  16.425000  
        2017-03-01      NaN            8.798738  18.4  16.062273  
        2017-04-01      NaN            7.223120  17.5  15.921667  
        2017-05-01      NaN           10.141096  17.0  16.104762  

[5 rows x 33 columns]

In [7]:
data.static_features.head()

,cat1,cat2,cat3,brand,sku_size,corr_20001,corr_20002,corr_20003,corr_20005,corr_20004,corr_20009,corr_20032,corr_20006,corr_20007,corr_20010
item_id,,,,,,,,,,,,,,,
20280,HC,ROPA LAVADO,Liquido,LIMPIEX,900,0.344376,0.251042,0.149971,0.056807,0.179661,0.107151,0.091045,-0.142871,0.511455,0.054308
20180,HC,ROPA LAVADO,Liquido,LIMPIEX,450,0.277107,0.164447,0.035649,0.454793,0.478442,0.356229,0.201214,-0.003315,0.772228,0.737458
20332,HC,ROPA LAVADO,Liquido,LIMPIEX,120,-0.620015,-0.316214,-0.103129,0.029887,0.088297,0.197564,0.070152,-0.304822,0.373020,0.510303
20222,HC,ROPA LAVADO,Liquido,LIMPIEX,450,0.403952,0.245037,0.294852,0.568689,0.475148,0.344290,0.123933,0.222539,0.442538,0.883702
20288,HC,ROPA LAVADO,Liquido,LIMPIEX,900,0.441335,0.129863,0.266306,0.254267,0.241912,0.499360,0.288560,-0.433734,0.179030,0.003566


In [8]:
covariates = data.drop(columns = "target").columns

predictor = TimeSeriesPredictor(target='target',
                                prediction_length=2, 
                                freq="M",
                                eval_metric = "WQL",
                                known_covariates_names = covariates).fit(data, 
                                                                         num_val_windows=3,
                                                                         val_step_size=1)

Frequency 'M' stored as 'ME'
Beginning AutoGluon training...
AutoGluon will save models to 'AutogluonModels\ag-20240706_192017'
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.11.2
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
CPU Count:          8
GPU Count:          0
Memory Avail:       1.51 GB / 7.88 GB (19.1%)
Disk Space Avail:   9.69 GB / 110.21 GB (8.8%)
	We recommend a minimum available disk space of 10 GB, and large datasets may require more.

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WQL,
 'freq': 'ME',
 'hyperparameters': 'default',
 'known_covariates_names': ['month',
                            'quarter',
                            'days_in_month',
                            'sundays',
                            'saturdays',
                            'lag1m',
                            'lag2m',
                            'lag3m',
                       

In [9]:
predictor.leaderboard()

,model,score_val,pred_time_val,fit_time_marginal,fit_order
0,WeightedEnsemble,-0.025021,18.663525,11.262417,12
1,DirectTabular,-0.025101,15.430153,5019.270383,3
2,RecursiveTabular,-0.138344,3.233372,1438.120919,2
3,PatchTST,-0.180033,1.104771,346.508639,11
4,AutoETS,-0.186423,23.330246,102.279073,7
5,DynamicOptimizedTheta,-0.191702,2.564576,81.482514,6
6,CrostonSBA,-0.200184,1.195610,42.035967,4
7,SeasonalNaive,-0.214376,1.615702,12.629532,1
8,AutoARIMA,-0.221880,65.872531,116.036097,8
9,NPTS,-0.260869,1.534147,3.918036,5


In [10]:
predictor.feature_importance()

Computing feature importance


,importance,stdev,n,p99_low,p99_high
cat1,0.000000,0.000000,5.0,0.000000,0.000000
cat2,0.000000,0.000000,5.0,0.000000,0.000000
cat3,0.000000,0.000000,5.0,0.000000,0.000000
brand,0.000000,0.000000,5.0,0.000000,0.000000
sku_size,-0.000019,0.000133,5.0,-0.000292,0.000255
corr_20001,-0.000027,0.000041,5.0,-0.000111,0.000057
corr_20002,0.000026,0.000367,5.0,-0.000729,0.000782
corr_20003,0.000161,0.000315,5.0,-0.000489,0.000810
corr_20005,0.000126,0.000298,5.0,-0.000488,0.000739
corr_20004,0.000352,0.000410,5.0,-0.000492,0.001196


In [11]:
known_covariates_future = df_related[df_related.timestamp > df.timestamp.max()]
# Llevamos las fechas futuras a fin de mes porque así lo espera el metodo predict
known_covariates_future['timestamp'] = pd.to_datetime(known_covariates_future['timestamp'])
known_covariates_future['timestamp'] = known_covariates_future['timestamp'] + pd.offsets.MonthEnd(0)
known_covariates_future['timestamp'] = known_covariates_future['timestamp'].dt.strftime('%Y-%m-%d')
print(known_covariates_future.timestamp.min(), known_covariates_future.timestamp.max())
known_covariates_future = TimeSeriesDataFrame.from_data_frame(known_covariates_future,
                                                              id_column="item_id",
                                                              timestamp_column="timestamp")
known_covariates_future.head()

2020-01-31 2020-02-29


month  quarter  days_in_month  sundays  saturdays  \
item_id timestamp                                                       
20001   2020-01-31      1        1             31        4          4   
        2020-02-29      2        1             29        4          5   
20002   2020-01-31      1        1             31        4          4   
        2020-02-29      2        1             29        4          5   
20003   2020-01-31      1        1             31        4          4   

                         lag1m       lag2m        lag3m        lag4m  \
item_id timestamp                                                      
20001   2020-01-31  1504.68856  1397.37231  1561.505520  1660.005610   
        2020-02-29         NaN  1504.68856  1397.372310  1561.505520   
20002   2020-01-31  1087.30855  1423.57739  1509.385713  1090.187710   
        2020-02-29         NaN  1087.30855  1423.577390  1509.385713   
20003   2020-01-31   892.50129   948.29393  1081.366450   967.771160   

                         lag5m  ...  delta6  delta7  delta8  delta9  delta10  \
item_id timestamp               ...                                            
20001   2020-01-31  1261.34529  ...     NaN     NaN     NaN     NaN      NaN   
        2020-02-29  1660.00561  ...     NaN     NaN     NaN     NaN      NaN   
20002   2020-01-31   813.78215  ...     NaN     NaN     NaN     NaN      NaN   
        2020-02-29  1090.18771  ...     NaN     NaN     NaN     NaN      NaN   
20003   2020-01-31   635.59563  ...     NaN     NaN     NaN     NaN      NaN   

                    delta11  delta12  monthly_percentage   ipc      dolar  
item_id timestamp                                                          
20001   2020-01-31      NaN      NaN            7.308178  40.0  77.477273  
        2020-02-29      NaN      NaN            7.212629  39.9  78.191667  
20002   2020-01-31      NaN      NaN            9.290639  40.0  77.477273  
        2020-02-29      NaN      NaN            7.649477  39.9  78.191667  
20003   2020-01-31      NaN      NaN           10.242019  40.0  77.477273  

[5 rows x 32 columns]

In [12]:
predictions = predictor.predict(data,
                                known_covariates = known_covariates_future)

data with frequency 'None' has been resampled to frequency 'ME'.
Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble


In [13]:
predictions_v1 = predictions.copy()
predictions_v1 = predictions_v1.reset_index()
predictions_v1 = predictions_v1[["item_id", "timestamp", "mean"]]
predictions_v1 = predictions_v1[predictions_v1.timestamp == "2020-02-29"]
predictions_v1 = predictions_v1.drop(columns = {"timestamp"})
predictions_v1 = predictions_v1.rename(columns = {"item_id":"product_id", "mean":"tn"})
predictions_v1.head(5)

,product_id,tn
1,20001,1401.976807
3,20002,1099.048340
5,20003,844.113220
7,20004,590.573792
9,20005,532.849854


In [14]:
predictions_v1.to_csv("C:/Users/usuario/otros/Desktop\MCD/6 Labo III/Forecasting Problem/Outputs/prediccion_autogluon_static_features_and_covariates_newFeatures.csv", sep = ",", index = False)

In [16]:
predictions = predictor.predict(data,
                                known_covariates = known_covariates_future,
                                model = "DirectTabular")

data with frequency 'None' has been resampled to frequency 'ME'.


In [17]:
predictions_v1 = predictions.copy()
predictions_v1 = predictions_v1.reset_index()
predictions_v1 = predictions_v1[["item_id", "timestamp", "mean"]]
predictions_v1 = predictions_v1[predictions_v1.timestamp == "2020-02-29"]
predictions_v1 = predictions_v1.drop(columns = {"timestamp"})
predictions_v1 = predictions_v1.rename(columns = {"item_id":"product_id", "mean":"tn"})
predictions_v1.head(5)

,product_id,tn
1,20001,1406.218506
3,20002,1103.453369
5,20003,846.968506
7,20004,593.018860
9,20005,535.524841


In [18]:
predictions_v1.to_csv("C:/Users/usuario/otros/Desktop\MCD/6 Labo III/Forecasting Problem/Outputs/prediccion_autogluon_static_features_and_covariates_newFeatures_DTM.csv", sep = ",", index = False)

In [19]:
predictions = predictor.predict(data,
                                known_covariates = known_covariates_future,
                                model = "RecursiveTabular")

data with frequency 'None' has been resampled to frequency 'ME'.


In [20]:
predictions_v1 = predictions.copy()
predictions_v1 = predictions_v1.reset_index()
predictions_v1 = predictions_v1[["item_id", "timestamp", "mean"]]
predictions_v1 = predictions_v1[predictions_v1.timestamp == "2020-02-29"]
predictions_v1 = predictions_v1.drop(columns = {"timestamp"})
predictions_v1 = predictions_v1.rename(columns = {"item_id":"product_id", "mean":"tn"})
predictions_v1.head(5)

,product_id,tn
1,20001,1155.957275
3,20002,843.560852
5,20003,678.508545
7,20004,448.760773
9,20005,377.703705


In [21]:
predictions_v1.to_csv("C:/Users/usuario/otros/Desktop\MCD/6 Labo III/Forecasting Problem/Outputs/prediccion_autogluon_static_features_and_covariates_newFeatures_RTM.csv", sep = ",", index = False)